In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math
import queue

In [2]:
iris = datasets.load_iris()

In [3]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [4]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [5]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [6]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [7]:
df.head()

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a


In [8]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [48]:
# Making a class for decision tree
class decisionTree:
    # A constructor to create the decison tree, takes the level as an argument
    def __init__(self,l):
        self.level = l
        self.entropy = -math.inf
        self.feature_to_be_split_upon = ""
        self.isLeaf = False
        self.count_op_classes = {}
        self.children = {}
    
    # A function to print the tree in level order. This function uses the queue data structure to print the nodes in 
    # level order.
    def print_tree(self):
        if self is None:
            return
        q = queue.Queue(maxsize = 50)
        t = 0
        q.put([t,self])
        t+=1
        while(not(q.empty())):
            curr = q.get()
            print("Node:", curr[0])
            print("Level of current node =", curr[1].level,end = " ")
            print("and entropy of current node =", curr[1].entropy)
            # If the node is a leaf node, print its output class
            if curr[1].isLeaf == True:
                op_count = curr[1].count_op_classes
                max_ans = 0
                ans = ""
                for i in op_count:
                    if op_count[i] > max_ans:
                        max_ans = op_count[i]
                        ans = i
                print("Leaf Node with output:",ans)
            # If the node is not leaf, print the feature on which it is split upon
            else:
                print("The feature upon which this node is split upon is:",curr[1].feature_to_be_split_upon)
                child_nodes = curr[1].children
                print("The children of current node according to the values of this feature are:")
                for i in child_nodes:
                    print("Node",t,":",i)
                    q.put([t,child_nodes[i]])
                    t+=1 
            print()        

In [10]:
# Requires y and logarithmic base as arguments to calculate the entropy of a node
def entropy(y,base):
    op = set(y[0])
    
    # Adding the count of each output class to my op_count list using the values_counts function of pandas dataframe
    op_count = []
    for i in op:
        curr_count = y[0].value_counts()[i]
        op_count.append(curr_count)
    
    # Applying the mathematical formula for entropy by traversing through op_count
    entropy_value = 0
    total_sum = sum(op_count)
    for i in op_count:
        entropy_value += (-1)*(i/total_sum)*math.log((i/total_sum),base)
    
    return entropy_value

In [11]:
# Requires x,y and the feature name to split the current node upon
def information_gain(x,y,feature_name):
    # information of current node
    curr_info = entropy(y,10)
    
    info_after_split = 0
    
    # different classes present in the current feature upon which the split is to made
    ip_classes = set(x[feature_name])
    for i in ip_classes:
        temp_x = x[x[feature_name] == i]
        temp_y = y[x[feature_name] == i]
        probability = len(temp_x)/len(x)
        info_after_split += entropy(temp_y,10)*probability
    
    info_gain = curr_info - info_after_split
    return info_gain

In [12]:
# caluclating the split info using the formula
def split_info(x,feature_name):
    split_info = 0
    ip_classes = set(x[feature_name])
    for i in ip_classes:
        temp_x = x[x[feature_name] == i]
        probability = len(temp_x)/len(x)
        split_info += (-1)*probability*math.log(probability,10)
    
    return split_info

In [13]:
#calculating the split ratio by dividing the information gain by split info for a particular node
def gain_ratio(x,y,feature_name):
    info_gain = information_gain(x,y,feature_name)
    si = split_info(x,feature_name)
    return info_gain/si

In [49]:
def build_tree(df, y, unused_features,level):
    # Create a node for current level using an object of decisionTree class
    node = decisionTree(level)
    
    # Print current level of the node
    print('Level', level)
    level += 1
    
    # Print the number of occurence of each output class for the current node and add them to the count dict of node 
    op_values = set(y[0])
    for i in [0,1,2]:
        if i not in op_values:
            print("Count of",i,"=",0)
            continue
        node.count_op_classes[i] = y[0].value_counts()[i]
        print("Count of",i,"=",y[0].value_counts()[i])
        
    # Print the current entropy of the node by calling the entropy function defined above
    ent = entropy(y,2)
    print("Entropy of current Node:", ent)
    node.entropy = ent
    
    #base case
    
    # 1. y contains only one distinct value
    if len(op_values) == 1:
        node.isLeaf = True
        print("Pure Leaf Node Reached")
        return node
    
    # 2. unused is empty
    if(len(unused_features)<=0):
        node.isLeaf = True
        print("No features Left to further split upon")
        print("Leaf Node Reached")
        return node
        
    
    # Iterating over the left unused features and using the gain ration function defined above to find the gain ratio
    # for each feature. The more the gain ratio, the better the split.
    x = df
    best_feature = ""
    max_gain_ratio = -math.inf
    for f in unused_features:
        curr_gain_ratio = gain_ratio(x,y,f)
        if curr_gain_ratio > max_gain_ratio:
            max_gain_ratio = curr_gain_ratio
            best_feature = f
        
        
    # here you should know the best feature
    # print it out and store that for our current node
    print("Splitting on feature:", best_feature)
    node.feature_to_be_split_upon = best_feature
    print("The Gain Ratio for this feature is:", max_gain_ratio)
    
    # remove best feature from unused features
    unused_features.remove(best_feature)
    
    # loop over possible values of best feature
    best_feature_values = set(x[best_feature])
    for i in best_feature_values:
        print()
        temp_x = x[x[best_feature] == i]
        temp_y = y[x[best_feature] == i]
        # call build tree recursively
        child = build_tree(temp_x, temp_y,unused_features,level)
        node.children[i] = child
    return node

In [52]:
# The code when run multiple times from scratch, gives the same output, just in different order. 
# To avoid any confusion, I am specifying the reason for this.
# Python sets and dictionaries are not ordered. Since sets and dicts are being used,
# it is possible that the left and right children on the same tree level may get swapped while splitting. The tree 
# formed is exactly the same in term of results. Just the order might be different. eg
#   3               3
#  / \     and     / \     are the same in terms of decision Trees.
# 1   2           2  1

In [50]:
y = pd.DataFrame(iris.target)
unused_features = set(df.columns)

# Printing done by the build_tree function which prints values at each node while also building the tree
print("Printing here is done by the build_tree function")
print()
root = build_tree(df, y, unused_features,0)

Printing here is done by the build_tree function

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Entropy of current Node: 1.584962500721156
Splitting on feature: pw_labeled
The Gain Ratio for this feature is: 0.6996382036222092

Level 1
Count of 0 = 50
Count of 1 = 0
Count of 2 = 0
Entropy of current Node: 0.0
Pure Leaf Node Reached

Level 1
Count of 0 = 0
Count of 1 = 40
Count of 2 = 16
Entropy of current Node: 0.863120568566631
Splitting on feature: pl_labeled
The Gain Ratio for this feature is: 0.43340994956210654

Level 2
Count of 0 = 0
Count of 1 = 39
Count of 2 = 8
Entropy of current Node: 0.6581912658132185
Splitting on feature: sl_labeled
The Gain Ratio for this feature is: 0.12674503775809323

Level 3
Count of 0 = 0
Count of 1 = 0
Count of 2 = 1
Entropy of current Node: 0.0
Pure Leaf Node Reached

Level 3
Count of 0 = 0
Count of 1 = 23
Count of 2 = 7
Entropy of current Node: 0.783776947484701
Splitting on feature: sw_labeled
The Gain Ratio for this feature is: 0.07092

In [53]:
# Printing is done using the stored tree
print("Printing in the level order format from the stored tree created using the decisionTree class")
print()
root.print_tree()

Printing in the level order format from the stored tree created using the decisionTree class

Node: 0
Level of current node = 0 and entropy of current node = 1.584962500721156
The feature upon which this node is split upon is: pw_labeled
The children of current node according to the values of this feature are:
Node 1 : a
Node 2 : c
Node 3 : b
Node 4 : d

Node: 1
Level of current node = 1 and entropy of current node = 0.0
Leaf Node with output: 0

Node: 2
Level of current node = 1 and entropy of current node = 0.863120568566631
The feature upon which this node is split upon is: pl_labeled
The children of current node according to the values of this feature are:
Node 5 : c
Node 6 : b
Node 7 : d

Node: 3
Level of current node = 1 and entropy of current node = 0.0
Leaf Node with output: 1

Node: 4
Level of current node = 1 and entropy of current node = 0.0
Leaf Node with output: 2

Node: 5
Level of current node = 2 and entropy of current node = 0.6581912658132185
The feature upon which thi